# 02 - Modelo Logístico Para Dataset

En este notebook se entrena un modelo base de **Regresión Logística** utilizando los datos ya procesados en `01_exploracion_datos_v2.ipynb`.


In [50]:
# Librerías principales
import pandas as pd
import numpy as np

# Librerías de modelado
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

# Configuración general
pd.set_option('display.max_columns', None)


**Modelo entrenado con Dataset rfm_clients_v3**

In [51]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

rfm_clients_v3 = pd.read_csv('../data/raw/clients_202510280926.csv')

# solo para probar estructura
print(rfm_clients_v3.head())


                              client_id      first_name last_name gender  age  \
0  21e5c13d-1c9a-4d00-9164-b72302d5edef         Tristán     Rojas      M   42   
1  36e48bdd-db11-4abe-9526-cfc90e68924d           Óscar  Barranco      M   58   
2  145c22df-3579-412e-bc12-b4fce70abaf3         Macario    Querol      M   28   
3  90c4a925-e51f-4dac-9193-2d9aec97a472          Ramiro   Sanchez      M   19   
4  853f711a-4c36-40b4-b5d0-6207152cd793  Juan Francisco      Flor      M   36   

                               email      city               address  \
0        tristán.rojas13@example.com    Rivera  Bulevar Artigas 3757   
1       óscar.barranco13@example.com     Salto  Bulevar Artigas 3911   
2       macario.querol67@example.com    Rivera        Av. Rivera 206   
3       ramiro.sanchez53@example.com     Salto  Av. 18 de Julio 6324   
4  juan francisco.flor68@example.com  Mercedes  Av. 18 de Julio 6301   

   postal_code  country citizenship document_type document_number  \
0        28

In [52]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

rfm_clients_v3 = pd.read_csv('../data/raw/clients_202510280926.csv')

# solo para probar estructura
print(rfm_clients_v3.head())


                              client_id      first_name last_name gender  age  \
0  21e5c13d-1c9a-4d00-9164-b72302d5edef         Tristán     Rojas      M   42   
1  36e48bdd-db11-4abe-9526-cfc90e68924d           Óscar  Barranco      M   58   
2  145c22df-3579-412e-bc12-b4fce70abaf3         Macario    Querol      M   28   
3  90c4a925-e51f-4dac-9193-2d9aec97a472          Ramiro   Sanchez      M   19   
4  853f711a-4c36-40b4-b5d0-6207152cd793  Juan Francisco      Flor      M   36   

                               email      city               address  \
0        tristán.rojas13@example.com    Rivera  Bulevar Artigas 3757   
1       óscar.barranco13@example.com     Salto  Bulevar Artigas 3911   
2       macario.querol67@example.com    Rivera        Av. Rivera 206   
3       ramiro.sanchez53@example.com     Salto  Av. 18 de Julio 6324   
4  juan francisco.flor68@example.com  Mercedes  Av. 18 de Julio 6301   

   postal_code  country citizenship document_type document_number  \
0        28

In [53]:
df = rfm_clients_v3.copy()
df['registration_date'] = pd.to_datetime(df['registration_date'], errors='coerce')
df['last_seen'] = pd.to_datetime(df['last_seen'], errors='coerce')
today = pd.Timestamp('today').normalize()
df['tenure_days'] = (today - df['registration_date']).dt.days
df['days_since_last_seen'] = (today - df['last_seen']).dt.days
df['churn'] = (df['days_since_last_seen'] >= 60).astype(int)

numeric_cols = ['age','tenure_days','days_since_last_seen']
categorical_cols = ['gender','city']

base = df[numeric_cols + categorical_cols + ['churn']].dropna()
X = pd.get_dummies(base[numeric_cols + categorical_cols], drop_first=True)
y = base['churn']


In [54]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

X_train_v3, X_test_v3, y_train_v3, y_test_v3 = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train_v3.shape, X_test_v3.shape, y_train_v3.mean(), y_test_v3.mean()


((1885, 12), (472, 12), 1.0, 1.0)

In [55]:
y_train_v3.value_counts()


churn
1    1885
Name: count, dtype: int64

In [56]:
y_test_v3.value_counts()


churn
1    472
Name: count, dtype: int64

In [57]:
df['churn'] = (df['days_since_last_seen'] >= 30).astype(int)


In [58]:
base = df[numeric_cols + categorical_cols + ['churn']].dropna()


In [59]:
y_train_v3.value_counts()


churn
1    1885
Name: count, dtype: int64

In [60]:
df['days_since_last_seen'].describe()


count    2357.000000
mean      314.456937
std       215.290005
min        68.000000
25%       135.000000
50%       256.000000
75%       442.000000
max      1030.000000
Name: days_since_last_seen, dtype: float64

In [61]:
for t in [10, 20, 30, 45, 60, 90]:
    churn_temp = (df['days_since_last_seen'] >= t).astype(int)
    print(t, churn_temp.value_counts(normalize=True).to_dict())


10 {1: 1.0}
20 {1: 1.0}
30 {1: 1.0}
45 {1: 1.0}
60 {1: 1.0}
90 {1: 0.8879932117098006, 0: 0.1120067882901994}


In [62]:
df[['registration_date','last_seen']].agg(['min','max'])
df['days_since_last_seen'].describe()
for t in [60,75,90,120,180,365]:
    c = (df['days_since_last_seen'] >= t).astype(int)
    print(t, c.value_counts().to_dict())


60 {1: 2357}
75 {1: 2259, 0: 98}
90 {1: 2093, 0: 264}
120 {1: 1862, 0: 495}
180 {1: 1523, 0: 834}
365 {0: 1544, 1: 813}


In [63]:
df['churn'] = (df['days_since_last_seen'] >= 90).astype(int)
df['churn'].value_counts()


churn
1    2093
0     264
Name: count, dtype: int64

In [64]:
numeric_cols = ['age','tenure_days','days_since_last_seen']
categorical_cols = ['gender','city']

base = df[numeric_cols + categorical_cols + ['churn']].dropna()
X = pd.get_dummies(base[numeric_cols + categorical_cols], drop_first=True)
y = base['churn']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

from sklearn.model_selection import train_test_split
X_train_v3, X_test_v3, y_train_v3, y_test_v3 = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

y_train_v3.value_counts(), y_test_v3.value_counts()


(churn
 1    1674
 0     211
 Name: count, dtype: int64,
 churn
 1    419
 0     53
 Name: count, dtype: int64)

In [65]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced', random_state=42)
log_model.fit(X_train_v3, y_train_v3)


LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [66]:
X_train_v3.to_csv('../data/processed/X_train_v3.csv', index=False)
X_test_v3.to_csv('../data/processed/X_test_v3.csv', index=False)
y_train_v3.to_csv('../data/processed/y_train_v3.csv', index=False)
y_test_v3.to_csv('../data/processed/y_test_v3.csv', index=False)


**Modelo entrenado con Dataset RFM**

In [67]:
import pandas as pd, numpy as np
X_train_v3 = pd.read_csv('../data/processed/X_train_v3.csv')
X_test_v3  = pd.read_csv('../data/processed/X_test_v3.csv')
y_train_v3 = pd.read_csv('../data/processed/y_train_v3.csv').squeeze()
y_test_v3  = pd.read_csv('../data/processed/y_test_v3.csv').squeeze()

X_train_v3.shape, X_test_v3.shape, y_train_v3.value_counts().to_dict(), y_test_v3.value_counts().to_dict()


((1885, 12), (472, 12), {1: 1674, 0: 211}, {1: 419, 0: 53})

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, average_precision_score

log_model = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced', random_state=42)
log_model.fit(X_train_v3, y_train_v3)

y_pred = log_model.predict(X_test_v3)
y_proba = log_model.predict_proba(X_test_v3)[:,1]

print(f'Accuracy: {accuracy_score(y_test_v3, y_pred):.3f}')
print(f'ROC AUC: {roc_auc_score(y_test_v3, y_proba):.3f}')
print(f'Average Precision: {average_precision_score(y_test_v3, y_proba):.3f}')
print(classification_report(y_test_v3, y_pred, digits=3))
confusion_matrix(y_test_v3, y_pred)


Accuracy: 0.913
ROC AUC: 0.998
Average Precision: 1.000
              precision    recall  f1-score   support

           0      0.564     1.000     0.721        53
           1      1.000     0.902     0.949       419

    accuracy                          0.913       472
   macro avg      0.782     0.951     0.835       472
weighted avg      0.951     0.913     0.923       472



array([[ 53,   0],
       [ 41, 378]])

In [69]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, classification_report, confusion_matrix

tree_model = DecisionTreeClassifier(max_depth=5, random_state=42, class_weight='balanced')
tree_model.fit(X_train_v3, y_train_v3)

y_pred_tree = tree_model.predict(X_test_v3)
y_proba_tree = tree_model.predict_proba(X_test_v3)[:,1]

print(f'Accuracy: {accuracy_score(y_test_v3, y_pred_tree):.3f}')
print(f'ROC AUC: {roc_auc_score(y_test_v3, y_proba_tree):.3f}')
print(f'Average Precision: {average_precision_score(y_test_v3, y_proba_tree):.3f}')
print(classification_report(y_test_v3, y_pred_tree, digits=3))
print(confusion_matrix(y_test_v3, y_pred_tree))


Accuracy: 1.000
ROC AUC: 1.000
Average Precision: 1.000
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        53
           1      1.000     1.000     1.000       419

    accuracy                          1.000       472
   macro avg      1.000     1.000     1.000       472
weighted avg      1.000     1.000     1.000       472

[[ 53   0]
 [  0 419]]


In [70]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, class_weight='balanced')
rf_model.fit(X_train_v3, y_train_v3)

y_pred_rf = rf_model.predict(X_test_v3)
y_proba_rf = rf_model.predict_proba(X_test_v3)[:,1]

print(f'Accuracy: {accuracy_score(y_test_v3, y_pred_rf):.3f}')
print(f'ROC AUC: {roc_auc_score(y_test_v3, y_proba_rf):.3f}')
print(f'Average Precision: {average_precision_score(y_test_v3, y_proba_rf):.3f}')
print(classification_report(y_test_v3, y_pred_rf, digits=3))
print(confusion_matrix(y_test_v3, y_pred_rf))


Accuracy: 1.000
ROC AUC: 1.000
Average Precision: 1.000
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        53
           1      1.000     1.000     1.000       419

    accuracy                          1.000       472
   macro avg      1.000     1.000     1.000       472
weighted avg      1.000     1.000     1.000       472

[[ 53   0]
 [  0 419]]


**Modelo entrenado con Dataset rfm_clients_v2**

In [73]:
rfm_clients_v2 = pd.read_csv('../data/raw/rfm_clients_v2.csv')
rfm_clients_v2.head()


,client_id,total_spent,num_orders,avg_order_val,first_purchase,last_purchase,num_categories,recency_days,frequency,monetary,customer_age_days,recency_segment,frequency_segment,monetary_segment,is_high_spender,is_inactive,is_frequent,avg_spent_per_category,orders_per_month,rfm_score,churn
0,00094d4c-1acd-46ec-a6d2-c35710c31005,133.85,2,66.925000,2023-11-17 03:46:17,2024-04-27 17:58:43,2,248,2,133.85,162,Muy inactivo,Baja,Bajo,0,1,0,44.616667,0.312500,3,1
1,0016d161-d6a2-42f3-bfd6-66aee6d163e3,235.11,4,58.777500,2023-01-03 14:33:10,2023-07-11 15:17:43,2,539,4,235.11,189,Muy inactivo,Baja,Bajo,0,1,0,78.370000,0.547945,3,1
2,0042e01f-9154-45fd-9c1d-499b3eb3f1c4,3943.76,77,51.217662,2023-01-16 05:43:41,2024-12-28 16:17:46,5,3,77,3943.76,712,Muy reciente,Muy alta,Premium,1,0,1,657.293333,3.113208,15,0
3,0052b716-46cf-4540-aa23-409642dc4048,3590.10,67,53.583582,2023-01-12 04:44:03,2024-12-28 12:43:15,5,3,67,3590.10,716,Muy reciente,Muy alta,Premium,1,0,1,598.350000,2.694370,15,0
4,00679c7c-bd80-4a70-9052-830bc5b63ecf,4489.99,79,56.835316,2023-01-30 09:58:29,2024-12-17 22:38:56,5,14,79,4489.99,687,Reciente,Muy alta,Premium,1,0,1,748.331667,3.305439,13,0


In [72]:
rfm_clients_v2 = pd.read_csv('../data/raw/rfm_clients_v2.csv')


In [76]:
import pandas as pd
import numpy as np

X_train_v2 = pd.read_csv('../data/processed/X_train_v2.csv')
X_test_v2  = pd.read_csv('../data/processed/X_test_v2.csv')
y_train_v2 = pd.read_csv('../data/processed/y_train_v2.csv').squeeze()
y_test_v2  = pd.read_csv('../data/processed/y_test_v2.csv').squeeze()

print("Shapes:", X_train_v2.shape, X_test_v2.shape)
print("y_train_v2:", y_train_v2.value_counts().to_dict())
print("y_test_v2:", y_test_v2.value_counts().to_dict())


Shapes: (1804, 25) (451, 25)
y_train_v2: {0: 1439, 1: 365}
y_test_v2: {0: 360, 1: 91}


In [78]:
import pandas as pd
import numpy as np

X_train_v2 = pd.read_csv('../data/processed/X_train_v2.csv')
X_test_v2  = pd.read_csv('../data/processed/X_test_v2.csv')
y_train_v2 = pd.read_csv('../data/processed/y_train_v2.csv').squeeze()
y_test_v2  = pd.read_csv('../data/processed/y_test_v2.csv').squeeze()

print("Shapes originales:", X_train_v2.shape, X_test_v2.shape)

# 🔴 NUEVO: quedarnos solo con columnas numéricas
num_cols_v2 = X_train_v2.select_dtypes(include='number').columns
print("Columnas numéricas V2:", num_cols_v2.tolist())

X_train_v2 = X_train_v2[num_cols_v2]
X_test_v2  = X_test_v2[num_cols_v2]

print("Shapes solo numéricas:", X_train_v2.shape, X_test_v2.shape)
print("y_train_v2:", y_train_v2.value_counts().to_dict())
print("y_test_v2:", y_test_v2.value_counts().to_dict())


Shapes originales: (1804, 25) (451, 25)
Columnas numéricas V2: ['total_spent', 'num_orders', 'avg_order_val', 'num_categories', 'recency_days', 'frequency', 'monetary', 'customer_age_days', 'is_high_spender', 'is_inactive', 'is_frequent', 'avg_spent_per_category', 'orders_per_month', 'rfm_score']
Shapes solo numéricas: (1804, 14) (451, 14)
y_train_v2: {0: 1439, 1: 365}
y_test_v2: {0: 360, 1: 91}


In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, roc_auc_score, average_precision_score,
    classification_report, confusion_matrix
)

log_model_v2 = LogisticRegression(
    solver='lbfgs',
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)

log_model_v2.fit(X_train_v2, y_train_v2)

y_pred_log_v2 = log_model_v2.predict(X_test_v2)
y_proba_log_v2 = log_model_v2.predict_proba(X_test_v2)[:,1]

print("=== Logistic Regression - RFM_V2 ===")
print("Accuracy:", round(accuracy_score(y_test_v2, y_pred_log_v2), 3))
print("ROC AUC:", round(roc_auc_score(y_test_v2, y_proba_log_v2), 3))
print("Average Precision:", round(average_precision_score(y_test_v2, y_proba_log_v2), 3))
print(classification_report(y_test_v2, y_pred_log_v2, digits=3))
print(confusion_matrix(y_test_v2, y_pred_log_v2))


=== Logistic Regression - RFM_V2 ===
Accuracy: 0.989
ROC AUC: 1.0
Average Precision: 0.998
              precision    recall  f1-score   support

           0      0.997     0.989     0.993       360
           1      0.957     0.989     0.973        91

    accuracy                          0.989       451
   macro avg      0.977     0.989     0.983       451
weighted avg      0.989     0.989     0.989       451

[[356   4]
 [  1  90]]


In [80]:
from sklearn.tree import DecisionTreeClassifier

tree_model_v2 = DecisionTreeClassifier(
    max_depth=5,
    random_state=42,
    class_weight='balanced'
)

tree_model_v2.fit(X_train_v2, y_train_v2)

y_pred_tree_v2 = tree_model_v2.predict(X_test_v2)
y_proba_tree_v2 = tree_model_v2.predict_proba(X_test_v2)[:,1]

print("=== Decision Tree - RFM_V2 ===")
print("Accuracy:", round(accuracy_score(y_test_v2, y_pred_tree_v2), 3))
print("ROC AUC:", round(roc_auc_score(y_test_v2, y_proba_tree_v2), 3))
print("Average Precision:", round(average_precision_score(y_test_v2, y_proba_tree_v2), 3))
print(classification_report(y_test_v2, y_pred_tree_v2, digits=3))
print(confusion_matrix(y_test_v2, y_pred_tree_v2))


=== Decision Tree - RFM_V2 ===
Accuracy: 1.0
ROC AUC: 1.0
Average Precision: 1.0
              precision    recall  f1-score   support

           0      1.000     1.000     1.000       360
           1      1.000     1.000     1.000        91

    accuracy                          1.000       451
   macro avg      1.000     1.000     1.000       451
weighted avg      1.000     1.000     1.000       451

[[360   0]
 [  0  91]]


In [81]:
from sklearn.ensemble import RandomForestClassifier

rf_model_v2 = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42,
    class_weight='balanced'
)

rf_model_v2.fit(X_train_v2, y_train_v2)

y_pred_rf_v2 = rf_model_v2.predict(X_test_v2)
y_proba_rf_v2 = rf_model_v2.predict_proba(X_test_v2)[:,1]

print("=== Random Forest - RFM_V2 ===")
print("Accuracy:", round(accuracy_score(y_test_v2, y_pred_rf_v2), 3))
print("ROC AUC:", round(roc_auc_score(y_test_v2, y_proba_rf_v2), 3))
print("Average Precision:", round(average_precision_score(y_test_v2, y_proba_rf_v2), 3))
print(classification_report(y_test_v2, y_pred_rf_v2, digits=3))
print(confusion_matrix(y_test_v2, y_pred_rf_v2))


=== Random Forest - RFM_V2 ===
Accuracy: 1.0
ROC AUC: 1.0
Average Precision: 1.0
              precision    recall  f1-score   support

           0      1.000     1.000     1.000       360
           1      1.000     1.000     1.000        91

    accuracy                          1.000       451
   macro avg      1.000     1.000     1.000       451
weighted avg      1.000     1.000     1.000       451

[[360   0]
 [  0  91]]
